# Testing Regression-based SysID
The goal here is to verify that we are able to estimate linear system dynamics from measurements

In [1]:
import tqdm

import numpy as np
import jax.numpy as jnp

from extravaganza.dynamical_systems import LDS
from extravaganza.sysid import Lifter

# seeds for randomness. setting to `None` uses random seeds
SYSTEM_SEED = None
SYSID_SEED = None

In [2]:
num_trials = 50
for _ in tqdm.trange(num_trials):
    # make system
    du = 20  # control dim
    ds = 30  # state dim
    system = LDS(ds, du, 'none', 'quad', seed=None)
    A, B = system.A, system.B

    # make sysid
    T0 = 200
    sysid = Lifter(obs_dim=ds, control_dim=du, state_dim=ds, max_traj_len=int(1e6), exploration_scale=0.5, method='identity', seed=SYSID_SEED)

    # interaction loop
    control = jnp.zeros(du)
    for _ in range(T0):
        cost, obs = system.interact(control)
        control = sysid.explore(cost, obs)
    Ahat, Bhat = sysid.end_exploration(wordy=False)

    atol = 1e-6
    assert jnp.allclose(A, Ahat, atol=atol), jnp.linalg.norm(A - Ahat)
    assert jnp.allclose(B, Bhat, atol=atol), jnp.linalg.norm(B - Bhat)
print('woohoo!')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:19<00:00,  2.58it/s]

woohoo!
